In [1]:
from pyoculus.problems import CylindricalBfield, AnalyticCylindricalBfield
from pyoculus.solvers import PoincarePlot, FixedPoint, Manifold
import matplotlib.pyplot as plt
import numpy as np

In [2]:
separatrix = {"type": "circular-current-loop", "amplitude": -4.2, "R": 3, "Z": -2.2}
# ps_cyl = AnalyticCylindricalBfield.without_axis(3, 0, 0.91, 0.7, perturbations_args = [separatrix], Rbegin = 1, Rend = 5, niter = 800, guess=[3.,-0.1],  tol = 1e-9)
ps_cyl = AnalyticCylindricalBfield(3, 0, 0.91, 0.7, perturbations_args = [separatrix])

In [12]:
from pyoculus.problems import CylindricalBfield, CartesianBfield
import functools
import copy


def cartesianize_B(B):
    @functools.wraps(B)
    def wrapper(self, xyz, *args, **kwargs):
        r = np.linalg.norm(xyz[0:2])
        phi = np.arctan2(xyz[1], xyz[0])
        invjac = CartesianBfield._inv_Jacobian(r, phi, xyz[2])
        # return np.linalg.inv(jac) @ B([r, phi, xyz[2]], *args, **kwargs)
        return np.linalg.solve(invjac, B([r, phi, xyz[2]], *args, **kwargs))
    return wrapper

def cartesianize_dBdX(dBdX):
    @functools.wraps(dBdX)
    def wrapper(self, xyz, *args, **kwargs):
        r = np.linalg.norm(xyz[0:2])
        phi = np.arctan2(xyz[1], xyz[0])
        rphiz = np.array([r, phi, xyz[2]])
        print("rphiz", rphiz)
        invjac = CartesianBfield._inv_Jacobian(*rphiz)
        jac = np.linalg.inv(invjac)
        b, dbdx = dBdX(rphiz, *args, **kwargs)
        b = b[0]
        print("dbdx", dbdx)
        chris = np.array([
            [0, -b[1]*r, 0 ],
            [b[1]/r, b[0]/r, 0],
            [0, 0, 0]
        ])
        print("chris", chris)
        b = (jac @ b.reshape(3,1)).flatten()
        return [b], jac @ (dbdx + chris) @ invjac
    return wrapper

from types import MethodType

def tocartesian(cylindricalbfield):
    return_class = copy.deepcopy(cylindricalbfield)

    return_class.__class__ = CartesianBfield
    return_class.B = MethodType(cartesianize_B(copy.deepcopy(cylindricalbfield.B)), return_class)
    return_class.dBdX = MethodType(cartesianize_dBdX(copy.deepcopy(cylindricalbfield.dBdX)), return_class)

    def f_RZ_wrapper(self, *args, **kwargs):
        return CartesianBfield.f_RZ(self, *args, **kwargs)
    def f_RZ_tangent_wrapper(self, *args, **kwargs):
        return CartesianBfield.f_RZ_tangent(self, *args, **kwargs)
    
    return_class.f_RZ = MethodType(f_RZ_wrapper, return_class)
    return_class.f_RZ_tangent = MethodType(f_RZ_tangent_wrapper, return_class)

    return return_class

In [13]:
ps_cart = tocartesian(ps_cyl)

In [14]:
ps_cart.f_RZ(0, [3., 0.1])

array([-0.43540489, -0.22731447])

In [15]:
ps_cyl.f_RZ(0, [3., 0.1])

array([-0.43540489, -0.22731447])

In [16]:
ps_cart.f_RZ_tangent(0., [3., 0.1, 1, 0, 0, 1])

xyz [3.  0.  0.1]
<bound method AnalyticCylindricalBfield.dBdX of <pyoculus.problems.cartesian_bfield.CartesianBfield object at 0x00000294215B3610>>
rphiz [3.  0.  0.1]
dbdx [[ 0.03649676  0.         -0.53054553]
 [-0.40732907  0.          0.0864851 ]
 [ 0.8027878   0.          0.05217977]]
chris [[ 0.         -1.83298083  0.        ]
 [ 0.20366454 -0.08867653  0.        ]
 [ 0.          0.          0.        ]]
dBdX [[ 0.03649676 -0.61099361 -0.53054553]
 [-0.61099361 -0.08867653  0.25945529]
 [ 0.8027878   0.          0.05217977]]
dBdX [[ 0.03649676 -0.61099361 -0.53054553]
 [-0.61099361 -0.08867653  0.25945529]
 [ 0.8027878   0.          0.05217977]]
dRphiZ [-0.2660296   0.61099361 -0.13888769]
dBdRphiZ [[ 0.03649676 -1.83298083 -0.53054553]
 [-0.20366454 -0.08867653  0.0864851 ]
 [ 0.8027878   0.          0.05217977]]
dBdRphiZ [[ 3.64967648e-02  2.22044605e-16 -5.30545529e-01]
 [-4.07329073e-01  0.00000000e+00  8.64850977e-02]
 [ 8.02787804e-01  0.00000000e+00  5.21797695e-02]]
M [

array([-0.43540489, -0.22731447, -0.23053646,  1.16236242, -0.80670156,
        0.11757747])

In [17]:
ps_cyl.f_RZ_tangent(0., [3., 0.1, 1, 0, 0, 1])

<bound method AnalyticCylindricalBfield.dBdX of <pyoculus.problems.toybox.AnalyticCylindricalBfield object at 0x00000294210D7F40>>
dBdRphiZ [[ 0.03649676  0.         -0.53054553]
 [-0.40732907  0.          0.0864851 ]
 [ 0.8027878   0.          0.05217977]]
dRphiZ [-0.2660296   0.61099361 -0.13888769]
M [[-0.23053646 -0.80670156]
 [ 1.16236242  0.11757747]]
dRZ [[1. 0.]
 [0. 1.]]
dRZ [[-0.23053646 -0.80670156]
 [ 1.16236242  0.11757747]]


array([-0.43540489, -0.22731447, -0.23053646,  1.16236242, -0.80670156,
        0.11757747])

In [ ]:
print(ps_cyl.dBdX)

In [ ]:
print(ps_cyl.f_RZ_tangent)

In [ ]:
print(ps_cart.B([3., 0., 0.1]))
print(ps_cart.dBdX([3., 0., 0.1])[0])

In [11]:
ps_cyl.B([3., 0., 0.1])

array([-0.2660296 ,  0.61099361, -0.13888769])

In [10]:
ps_cyl.dBdX([3., 0., 0.1])

([array([-0.2660296 ,  0.61099361, -0.13888769])],
 array([[ 0.03649676,  0.        , -0.53054553],
        [-0.40732907,  0.        ,  0.0864851 ],
        [ 0.8027878 ,  0.        ,  0.05217977]]))

In [ ]:
invjac = CartesianBfield._inv_Jacobian(3., 0., 0.1)

In [ ]:
b = ps_cyl.B([3., 0., 0.1])
chris = np.array([
    [0, -b[1]*3, 0 ],
    [b[1]/3, b[0]/3, 0],
    [0, 0, 0]
])

In [ ]:
ps_cyl.dBdX([3., 0., 0.1])[1]

In [ ]:
np.linalg.inv(invjac) @ (ps_cyl.dBdX([3., 0., 0.1])[1] + chris) @ invjac

In [ ]:
ps_cart.dBdX([3., 0., 0.1])

In [ ]:
invjac @ np.array(ps_cart.dBdX([3., 0., 0.1])[1]) @ np.linalg.inv(invjac) - chris

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-12

pparams = dict()
pparams["nrestart"] = 0
pparams['niter'] = 600

fp = FixedPoint(ps_cyl, pparams, integrator_params=iparams)

# fp.compute(guess=[3.117263523069049, -1.6173346133145015], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
fp.compute(guess=[3.1072023810385443, -1.655410284892828], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)

In [ ]:
results = [list(p) for p in zip(fp.x, fp.y, fp.z)]

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-10

# set up the Poincare plot
pparams = dict()
pparams["Rbegin"] = 3.01
pparams["Rend"] = 5.5
pparams["nPtrj"] = 20
pparams["nPpts"] = 200
pparams["zeta"] = 0
# pparams["Z"] = -6

# Set RZs
nfieldlines = pparams["nPtrj"]+1
n1, n2 = int(np.ceil(nfieldlines/2)), int(np.floor(nfieldlines/2))

# # Simple way from opoint to xpoint then to coilpoint
Rs = np.concatenate((np.linspace(ps._R0, results[0][0], n1), np.linspace(results[0][0], separatrix['R']-1e-4, n2)))
Zs = np.concatenate((np.linspace(ps._Z0, results[0][2], n1), np.linspace(results[0][2], separatrix['Z']-1e-4, n2)))
RZs = np.array([[r, z] for r, z in zip(Rs, Zs)])

In [ ]:
pplot_perturbed = PoincarePlot(ps, pparams, integrator_params=iparams)
pdata = pplot_perturbed.compute(RZs)